<a href="https://colab.research.google.com/github/Kaique-William/AgenteIA/blob/main/gente_de_estudo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

!pip install -q google-adk

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google import genai

client = genai.Client()

from IPython.display import HTML, Markdown

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

    # Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Pega o assunto --- #
##########################################

def agente_de_busca(topico, pontos, data_de_hoje):
    buscador = Agent(
        name="agente_de_busca",
        model="gemini-2.0-flash",
        description="Recolhe e inicia a busca de informações",
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisa. sua função é pegar as informações passadas e e buscar o que for mais importantes sobre o assunto
        utilizando o Google (google_search). busque as informações mais recentes e relevantes ao assunto, tendo foco principal em informações
        vindas de sites oficiais. um limite de até 3 topicos importantes, com base nas relevancias com o assunto, datas recentes e importancia para
        a situação. com base nos pontos passados procure definir o nivel de conchecimento que a pessoa tem do assunto, se aparentar ser alto procure ser mais
        aprofundado com as informaçoes buscadas. Se caso se mostar mais leiga, busque informações introdutorias para dar uma base.
        """
        )

    entrada_do_agente_de_busca = f"Tópico: {topico}\nPontos Relevantes: {pontos}\nData de hoje: {data_de_hoje}"

    lancamentos_buscados = call_agent(buscador, entrada_do_agente_de_busca)
    return lancamentos_buscados

In [ ]:
################################################
# --- Agente 2: questionador1 --- #
################################################

def agente_questionador1(topico, pontos, lancamentos_buscados):
    questionador1 = Agent(
        name="agente_questionador1",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Você é um especialista no assunto, e busca trazer as informações mais impostantes e atuais. tendo em base os topicos trazidos, veja quais os mais
        importantes e os mais atuais em relação com os pontos relevantes. para isso você deve usar o google (google_search) para tem mais profundidade com o assundo,
        que ira abosrdar.
        Você trabalhara junto do questionador2 para decidir quais serão os topicos a serem passados, havendo consenço, enviar "CONSENÇO" e abaixo sua resposta
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_questionador1 = f"Tópico:{topico}\nPntos Relevantes{pontos}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    pensamento1 = call_agent(questionador1, entrada_do_agente_questionador1)
    return pensamento1

In [ ]:
################################################
# --- Agente 3: questionador2 --- #
################################################

def agente_questionador2(topico, pontos, lancamentos_buscados):
    questionador2 = Agent(
        name="agente_questionador2",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Você é um especialista no assunto, e busca trazer as informações mais dinamicas e expesificas. tendo em base os topicos trazidos, veja quais os mais
        importantes e os mais importantes em relação com os pontos relevantes. para isso você deve usar o google (google_search) para tem mais profundidade com o assundo,
        que ira abosrdar.
        Você trabalhara junto do questionador1 para decidir quais serão os topicos a serem passados, havendo consenço, enviar "CONSENÇO" e abaixo sua resposta
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_questionador2 = f"Tópico:{topico}\nPontos Relevantes{pontos}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    pensamento2 = call_agent(questionador2, entrada_do_agente_questionador2)
    return pensamento2

In [ ]:
######################################
# --- Agente 4: revisor --- #
######################################
def agente_revisor(topico, pontos, pensamento1, pensamento2):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
              Você é um revisor criativo e empático. Sua missão é transformar dois raciocínios em um conteúdo claro, instigante e alinhado ao nível de conhecimento da pessoa.

              Você deve:
              - Identificar os pontos centrais e complementares de cada raciocínio.
              - Produzir um texto de apoio ou roteiro que conecte esses temas com fluidez.
              - Incluir ideias paralelas, conexões relevantes e sugestões de caminhos para quem quer aprender mais sobre o tema.
              - Evite parecer técnico ou robótico: escreva como alguém que realmente se importa em tornar o assunto interessante para quem vai ler.
        """,

        description="Agente revisor"
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nPontos relevantes: {pontos}\nScripts: {pensamento1, pensamento2 }"
    # Executa o agente
    rascunho = call_agent(revisor, entrada_do_agente_revisor)
    return rascunho

In [ ]:
##########################################
# --- Agente 5: instrutor --- #
##########################################
def agente_instrutor(topico, pontos, rascunho):
    instrutor = Agent(
        name="agente_instrutor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um instrutor cativante e didático. Sua missão é ensinar com leveza, empatia e flexibilidade, adaptando o conteúdo à idade,
            contexto e nível de conhecimento de quem aprende.

            O que você deve fazer:
            - Use um tom amigável e acessível, como se estivesse explicando para um colega ou aluno interessado.
            - Adapte a linguagem: use metáforas, comparações, analogias e exemplos reais conforme o perfil da pessoa.
            - Crie seções curtas e envolventes, que ajudem a absorver o conteúdo em partes.
            - Faça perguntas, sugira ideias extras e incentive a curiosidade.
            - Evite frases genéricas de "feedback automático". O objetivo é guiar, não avaliar.

            Ex: "Você já ouviu falar em X? Vamos ver como isso se conecta com o que estamos estudando…"
        """,
        description="Agente instrutor",

    )
    entrada_do_agente_instrutor = f"Tópico: {topico}\nPontos importantes: {pontos}\nRascunho: {rascunho}"
    # Executa o agente
    instrucao = call_agent(instrutor, entrada_do_agente_instrutor)
    return instrucao

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de estudo de assuntos 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite sobre o que entender e/ou estudar: ")
pontos = input("📝 Por favor, digite os PONTOS RELEVANTES sobre o qual você quer entender e/ou estudar: ")

# Inserir lógica do sistema de agentes ################################################

if not topico:
    print("❗ Não se esqueça de informar o tópico!")
else:
    print(f"OK, verei o que há de mais importante sobre '{topico}' e o que posso fazer para ajudar.")

    busca = agente_de_busca(topico, pontos, data_de_hoje)

    # Primeira rodada: cada agente gera resposta inicial
    questao1 = agente_questionador1(topico, pontos, busca)
    questao2 = agente_questionador2(topico, pontos, questao1)

    # Loop até ambos enviarem "CONSENÇO"
    while "CONSENÇO" not in questao1 or "CONSENÇO" not in questao2:
        print("\n⚠️ Alinhando pontos...\n")

        questao1 = agente_questionador1(topico, pontos, questao2)
        questao2 = agente_questionador2(topico, pontos, questao1)

    print("\n✅ CONSENÇO alcançado. Revisando.")

    texto_revisado = agente_revisor(topico, pontos, questao1, questao2)

    instrucao = agente_instrutor(topico, pontos, texto_revisado)
    display(to_markdown(instrucao))



🚀 Iniciando o Sistema de estudo de assuntos 🚀
❓ Por favor, digite sobre o que entender e/ou estudar: segunda guerra mundial
📝 Por favor, digite os PONTOS RELEVANTES sobre o qual você quer entender e/ou estudar: como começou e com teve seu fim?
OK, verei o que há de mais importante sobre 'segunda guerra mundial' e o que posso fazer para ajudar.

✅ CONSENÇO alcançado. Revisando.

--- Instruções geradas ---



> Olá! Que bom que você se interessou por esse tema tão importante e impactante! A Segunda Guerra Mundial é um evento complexo, mas vamos explorá-lo juntos de forma clara e interessante.
> 
> Seu rascunho está ótimo! Ele cobre os pontos principais de forma bem organizada e acessível. Para deixar ainda melhor, podemos adicionar alguns detalhes e nuances que podem enriquecer a compreensão do tema.
> 
> **1. Aprofundando as causas:**
> 
> Você mencionou o Tratado de Versalhes e a crise econômica como fatores que contribuíram para o surgimento de líderes autoritários. Podemos adicionar mais alguns elementos:
> 
> *   **Nacionalismo exacerbado:** Após a Primeira Guerra, muitos países cultivaram um forte sentimento de nacionalismo, o que levou a tensões e rivalidades.
> *   **Fracasso da Liga das Nações:** A Liga das Nações, criada para evitar novas guerras, mostrou-se ineficaz diante das ambições expansionistas de alguns países.
> *   **Política de apaziguamento:** A política de apaziguamento adotada por Reino Unido e França, que consistia em ceder às exigências de Hitler na esperança de evitar a guerra, acabou encorajando suas ambições.
> 
> **2. Detalhando o curso da guerra:**
> 
> Você mencionou a Blitzkrieg e a invasão da Polônia. Podemos adicionar alguns pontos importantes:
> 
> *   **A Batalha da Inglaterra:** A resistência britânica aos ataques aéreos alemães foi crucial para impedir a invasão da Grã-Bretanha.
> *   **A Operação Barbarossa:** A invasão da União Soviética pela Alemanha em 1941 abriu uma nova frente de batalha e mudou o curso da guerra.
> *   **O ataque a Pearl Harbor:** O ataque japonês à base naval americana em Pearl Harbor, em 1941, levou os Estados Unidos a entrarem na guerra.
> 
> **3. Refinando o fim da guerra:**
> 
> Você descreveu bem os eventos que levaram à rendição da Alemanha e do Japão. Podemos adicionar algumas informações:
> 
> *   **A Conferência de Yalta:** Nessa conferência, os líderes dos Aliados (Churchill, Roosevelt e Stalin) se reuniram para discutir o futuro da Europa após a guerra.
> *   **O impacto das bombas atômicas:** A decisão de usar as bombas atômicas é controversa até hoje. Alguns argumentam que foi necessária para evitar uma invasão terrestre do Japão, enquanto outros a consideram um crime de guerra.
> 
> **4. Explorando as consequências:**
> 
> Você mencionou o Holocausto, a Guerra Fria e a criação da ONU. Podemos adicionar:
> 
> *   **Os Julgamentos de Nuremberg:** Os líderes nazistas foram julgados por crimes de guerra e crimes contra a humanidade.
> *   **A Declaração Universal dos Direitos Humanos:** Adotada pela ONU em 1948, estabeleceu os direitos básicos que todos os seres humanos devem ter.
> *   **A divisão da Alemanha:** A Alemanha foi dividida em quatro zonas de ocupação (americana, britânica, francesa e soviética), o que levou à criação da Alemanha Ocidental e da Alemanha Oriental.
> 
> Para deixar o tema ainda mais interessante, que tal falarmos sobre a participação do Brasil na Segunda Guerra Mundial? Ou sobre a vida das pessoas durante o conflito?
